In [1]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [2]:
# If working in Colab, mount your drive
from google.colab import drive
drive.mount('/content/drive')

# Direct to your project folder.
%cd /content/drive/Othercomputers/Computer1/dl-robocop

Mounted at /content/drive
/content/drive/Othercomputers/Computer1/dl-robocop


In [3]:
# Installing requirements
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 56.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 

In [17]:
from datasets import load_dataset
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import evaluate
import gensim
import transformers
import nltk
import random
#import jsonlines

In [39]:
data_path = 'data/'
dataset = pd.read_csv(data_path + 'paradetox.tsv', sep='\t')

In [40]:
test_samples = dataset.iloc[1200:1210]
dataset = dataset.head(1000)

In [27]:
model_name = "vicgalle/gpt2-alpaca-gpt4"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(model_name)
MAX_SEQ_LENGTH = tokenizer.model_max_length

In [28]:
from torch.utils.data import TensorDataset, DataLoader

# Define the maximum length and other parameters
max_length = 36

# Generate the input_ids and input_mask for each sample
input_ids = [torch.tensor(tokenizer.encode(d, max_length=max_length, padding='max_length')) for d in dataset['toxic']]
detoxified_ids = [torch.tensor(tokenizer.encode(d, max_length=max_length, padding='max_length')) for d in dataset['neutral1']]
input_mask = [torch.tensor([1 if token != tokenizer.pad_token_id else 0 for token in tokens]) for tokens in input_ids]

# Convert lists to tensors
input_ids_tensor = torch.stack(input_ids)
detoxified_ids_tensor = torch.stack(detoxified_ids)
input_mask_tensor = torch.stack(input_mask)

# Create a TensorDataset
dataset = TensorDataset(input_ids_tensor, detoxified_ids_tensor, input_mask_tensor)

In [29]:
batch_size = 32
lr = 1e-5
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_epochs = 1

In [31]:
for epoch in range(num_epochs):
  model.train()
  total_loss = 0.0
  for i,batch in enumerate(dataloader):
    input_ids, detoxified_ids, input_mask = batch

    model_outputs = model(input_ids=input_ids, attention_mask=input_mask, labels=detoxified_ids)
    loss = model_outputs.loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    # Print average loss for the epoch
    print(f"Train Loss: {loss.item():.4f}")
  average_loss = total_loss / len(dataloader)
  print(f"Epoch {epoch+1}, Average Loss: {average_loss:.4f}")


Train Loss: 10.2882
Train Loss: 9.6240
Train Loss: 9.2656
Train Loss: 8.5633
Train Loss: 8.1424
Train Loss: 7.6346
Train Loss: 7.0775
Train Loss: 7.0625
Train Loss: 6.5217
Train Loss: 6.1345
Train Loss: 5.7747
Train Loss: 5.6319
Train Loss: 5.1128
Train Loss: 4.8510
Train Loss: 4.7847
Train Loss: 3.9672
Train Loss: 4.1133
Train Loss: 3.6300
Train Loss: 3.4004
Train Loss: 3.1923
Train Loss: 3.1422
Train Loss: 2.8551
Train Loss: 2.6599
Train Loss: 2.3238
Train Loss: 2.6036
Train Loss: 2.5442
Train Loss: 2.4412
Train Loss: 1.8643
Train Loss: 2.1754
Train Loss: 2.3395
Train Loss: 2.8254
Train Loss: 2.2949
Epoch 1, Average Loss: 4.8388
